In [7]:
import numpy as np
from itertools import product

In [2]:
radius   = 1737400
alt      = 2000000
ground   = 7.5
exposure = 0.005
samples  = 1000
lines    = 1000

In [30]:
sensor_rad = radius + alt
angle_per_line = ground / radius
angle_per_samp = angle_per_line
angle_per_Second = angle_per_line / exposure

In [4]:
positions = []
velocities = []
for time in np.arange(0, (lines+1) * exposure, 0.25):
    angle = angle_per_Second * time
    positions.append([np.cos(angle), 0, -np.sin(angle)])
    velocities.append([-np.sin(angle), 0, -np.cos(angle)])
positions = sensor_rad * np.array(positions)
velocities = sensor_rad * np.array(velocities)

In [5]:
print(positions)

[[ 3.73740000e+06  0.00000000e+00 -0.00000000e+00]
 [ 3.73739991e+06  0.00000000e+00 -8.06679515e+02]
 [ 3.73739965e+06  0.00000000e+00 -1.61335899e+03]
 [ 3.73739922e+06  0.00000000e+00 -2.42003839e+03]
 [ 3.73739861e+06  0.00000000e+00 -3.22671768e+03]
 [ 3.73739782e+06  0.00000000e+00 -4.03339682e+03]
 [ 3.73739687e+06  0.00000000e+00 -4.84007577e+03]
 [ 3.73739573e+06  0.00000000e+00 -5.64675450e+03]
 [ 3.73739443e+06  0.00000000e+00 -6.45343296e+03]
 [ 3.73739295e+06  0.00000000e+00 -7.26011112e+03]
 [ 3.73739129e+06  0.00000000e+00 -8.06678895e+03]
 [ 3.73738947e+06  0.00000000e+00 -8.87346640e+03]
 [ 3.73738746e+06  0.00000000e+00 -9.68014343e+03]
 [ 3.73738529e+06  0.00000000e+00 -1.04868200e+04]
 [ 3.73738294e+06  0.00000000e+00 -1.12934961e+04]
 [ 3.73738041e+06  0.00000000e+00 -1.21001717e+04]
 [ 3.73737771e+06  0.00000000e+00 -1.29068467e+04]
 [ 3.73737484e+06  0.00000000e+00 -1.37135211e+04]
 [ 3.73737179e+06  0.00000000e+00 -1.45201949e+04]
 [ 3.73736857e+06  0.00000000e+

In [6]:
print(velocities)

[[-0.00000000e+00  0.00000000e+00 -3.73740000e+06]
 [-8.06679515e+02  0.00000000e+00 -3.73739991e+06]
 [-1.61335899e+03  0.00000000e+00 -3.73739965e+06]
 [-2.42003839e+03  0.00000000e+00 -3.73739922e+06]
 [-3.22671768e+03  0.00000000e+00 -3.73739861e+06]
 [-4.03339682e+03  0.00000000e+00 -3.73739782e+06]
 [-4.84007577e+03  0.00000000e+00 -3.73739687e+06]
 [-5.64675450e+03  0.00000000e+00 -3.73739573e+06]
 [-6.45343296e+03  0.00000000e+00 -3.73739443e+06]
 [-7.26011112e+03  0.00000000e+00 -3.73739295e+06]
 [-8.06678895e+03  0.00000000e+00 -3.73739129e+06]
 [-8.87346640e+03  0.00000000e+00 -3.73738947e+06]
 [-9.68014343e+03  0.00000000e+00 -3.73738746e+06]
 [-1.04868200e+04  0.00000000e+00 -3.73738529e+06]
 [-1.12934961e+04  0.00000000e+00 -3.73738294e+06]
 [-1.21001717e+04  0.00000000e+00 -3.73738041e+06]
 [-1.29068467e+04  0.00000000e+00 -3.73737771e+06]
 [-1.37135211e+04  0.00000000e+00 -3.73737484e+06]
 [-1.45201949e+04  0.00000000e+00 -3.73737179e+06]
 [-1.53268679e+04  0.00000000e+

In [31]:
ground_points = np.zeros((int(lines/10) + 1, int(samples/10) + 1, 3))
counter = 0
for line, sample in product(np.arange(0, lines + 1, 10), np.arange(0, samples + 1, 10)):
    lat = -angle_per_line * line
    lon = angle_per_samp * sample
    sub_sc_point = [np.cos(lat) * np.cos(lon), np.cos(lat) * np.sin(lon), np.sin(lat)]
    ground_points[int(line/10), int(sample/10), :] = sub_sc_point
ground_points = radius * ground_points

In [32]:
print(ground_points[:, :, 0])

[[1737400.         1737399.9983812  1737399.99352481 ... 1737384.45308523
  1737384.13418295 1737383.8120431 ]
 [1737399.9983812  1737399.9967624  1737399.99190601 ... 1737384.45146645
  1737384.13256416 1737383.81042431]
 [1737399.99352481 1737399.99190601 1737399.98704961 ... 1737384.4466101
  1737384.12770781 1737383.80556796]
 ...
 [1737384.45308523 1737384.45146645 1737384.4466101  ... 1737368.90630959
  1737368.58741015 1737368.26527319]
 [1737384.13418295 1737384.13256416 1737384.12770781 ... 1737368.58741015
  1737368.26851078 1737367.94637387]
 [1737383.8120431  1737383.81042431 1737383.80556796 ... 1737368.26527319
  1737367.94637387 1737367.62423702]]


In [35]:
print(ground_points[50, 50])

[1737391.90602155    3749.98835331   -3749.99708833]


In [67]:
samp_range = np.arange(0, samples+0.00000001, 0.0001)

In [68]:
lons = samp_range * angle_per_samp
gps = radius * np.vstack([np.cos(lons), np.sin(lons), 0.0 * lons]).T
sensor_pos = positions[0]
slant_ranges = np.linalg.norm(gps - sensor_pos, axis=1)
ground_ranges = samp_range * ground

In [69]:
print(list(np.polynomial.polynomial.polyfit(ground_ranges, slant_ranges, 3)))

[2000000.0000039602, -1.0504347070801814e-08, 5.377926500384916e-07, -1.3072206620088014e-15]
